# Exercise 1.5.1 - Non-Maximum Suppression
#### By Jonathan L. Moran (jonathan.moran107@gmail.com)
From the Self-Driving Car Engineer Nanodegree programme offered by Udacity.

## Objectives

* Implement the Non-Maximum Suppression ([NMS](https://learnopencv.com/non-maximum-suppression-theory-and-implementation-in-pytorch/)) algorithm;
* Use the Intersection over Union ([IoU](https://en.wikipedia.org/wiki/Jaccard_index)) metric with a threshold value of $0.7$; 
* Apply the NMS algorithm to the provided frame from the [Waymo Open Dataset](https://waymo.com/open/);
* (Optional) Use the [Soft-NMS](https://arxiv.org/abs/1704.04503) algorithm to re-score the bounding box predictions.

## 1. Introduction

In [1]:
### Importing the required modules

In [2]:
import json
import numpy as np
import os
import matplotlib as mpl
from matplotlib import colors, patches
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from typing import List, Union

In [3]:
tf.__version__

'2.4.0'

In [4]:
tf.test.gpu_device_name()

2022-09-24 14:45:01.765625: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


''

In [5]:
### Setting the environment variables

In [6]:
ENV_COLAB = False                # True if running in Google Colab instance

In [7]:
# Root directory
DIR_BASE = '' if not ENV_COLAB else '/content/'

In [8]:
# Subdirectory to save output files
DIR_OUT = os.path.join(DIR_BASE, 'out/')
# Subdirectory pointing to input data
DIR_SRC = os.path.join(DIR_BASE, 'data/')

In [9]:
### Creating subdirectories (if not exists)
os.makedirs(DIR_OUT, exist_ok=True)

### 1.1. Non-Maximum Suppression

#### Background
* Why is it used?
* How is it implemented?
* What are the drawbacks?

[Non-maximum suppression](https://paperswithcode.com/method/non-maximum-suppression) (NMS) [1] is a popular technique used in object detection pipelines for handling duplicate or redundant bounding box predictions. Since object detection algorithms tend to produce more than one candidate bounding boxes for a single object, NMS is used to preserve only the _best_ bounding box per object using a overlap score. 

#### History

Traditional object detection algorithms used an _exhaustive search_ method; by iteratively spanning the entire image space, any and all objects could in theory be precisely located. While this sliding window approach was successful in detecting objects at all sorts of unpredictable locations, exhaustive search was an extremely expensive algorithm to run — exploring and eliminating tens of thousands of candidate regions per image was far from efficient. Exhaustive search and other sliding window-based algorithms quickly became a relic and were replaced with more intuitive algorithms for object detection.

Modern deep learning detection algorithms use a _region-based_ "approximation" algorithm to obtain object locations. Rather than scanning the entire image space iteratively, the image is split into sub-regions and analysed more efficiently using convolutional feature maps to determine whether or not an object is present. The [region-based convolutional neural networks](https://en.wikipedia.org/wiki/Region_Based_Convolutional_Neural_Networks) family, known as R-CNNs, revolutionised object detection and helped speed up efficiency to allow for real-time detection needed in today's applications of self-driving car technology. 

With these new methods brought a new set of challenges; R-CNN architectures often produced many candidate bounding boxes for each object detected. In order to eliminate the redundant boxes and preserve only one candidate bounding box per object, a new "post-processing" step needed to be defined. In 2009, Felzenszwalb et al., created _non-maxima suppression_ (NMS), a scoring metric that combined the [Intersection over Union](https://en.wikipedia.org/wiki/Jaccard_index) (IoU) and the predicted _confidence score_ into a single metric. NMS effectively discarded all but one bounding box for each object, eliminating the redundant bounding box problem.

### 1.2. Soft-NMS

Researchers began to notice that non-maximum suppression wasn't perfect; NMS tended to disregard otherwise valid objects of interest. These objects in particular were _occluded_ (obstructed or "blocked" by another object). Such objects have bound to have confidence scores that are lower than the average, thus a fixed threshold wouldn't cut it.

Simply decreasing the confidence threshold to account for this could lead to a drop in average precision, increasing the number of _false positives_ ("other" bounding boxes belonging to the same object). Consequently, when the overlap threshold is increased, valid bounding boxes can be unintentionally suppressed (discarded), leaving some objects without a corresponding bounding box. This is especially relevant in the driving environment. As in high-density traffic conditions; two separate cars might share a high-degree of overlap. As a result, only one of the two overlapping bounding boxes would be preserved (the one with the greatest confidence threshold). In other words, both cars would be incorrectly assigned to the same bounding box.

Soft-NMS by Bodla et al., seeks to address this problem in their cleverly-titled paper "Improving Object Detection With One Line of Code" [2]. Rather than suppressing boxes with a high-degree of overlap, we can instead decrease ("decay") their classification score. Re-running those candidates through a thresholding function could have one of two effects: those candidates no longer meet the threshold requirement and are therefore dropped, or, they are above the threshold and can therefore be kept. While the separate cars scenario might benefit from this simple case, as both boxes would be preserved, the more trivial case might be that the neighbouring redundant bounding box candidates would be incorrectly kept as well. Because of this likely scenario, Bodla et al. proposed a decay function that decreased the confidence scores linearly proportional to overlap amount. Therefore, candidate boxes that had a very high degree of overlap would experience a larger decay (greater penalty) than neighbouring bounding boxes without as much overlap. To better account for false positives, the Gaussian penalty function was introduced. By applying the update rule in each iteration, boxes with the highest degree of overlap are pruned and the number of false positives reduced at each step.

## 2. Programming Task

### 2.1. Non-Maximum Suppression

You are given a json file containing a list of predictions, containing `boxes` and `scores`.

You will leverage the `calculate_iou` function to calculate the Intersection Over Union (IoU) of these different predictions and implement the NMS algorithm.

#### IoU algorithm in Python

In [10]:
### From Udacity's `utils.py`

In [11]:
def calculate_iou(
        bboxes1: tf.Tensor, 
        bboxes2: tf.Tensor
) -> tf.Tensor:
    """Calculates the IoU score between a batch of bounding boxes.
    
    It is assumed that the number of bounding boxes N and M in each batch
    are equal. It is also assumed that the coordinates are ordered as follows:
        bbox := [y1, x1, y2, x2].
    
    :param bboxes1: the Nx4 bounding box coordinates.
    :param bboxes2: the Mx4 bounding box coordinates.
    :returns: iou, the intersection over union (IoU) scores between
        the two batches of bounding boxes.
    """
    
    ### 1. Get the upper and lower (y, x) coordinate pairs
    ymin = tf.math.reduce_max([bboxes1[..., 0], bboxes2[..., 0]])
    xmin = tf.math.reduce_max([bboxes1[..., 1], bboxes2[..., 1]])
    ymax = tf.math.reduce_min([bboxes1[..., 2], bboxes2[..., 2]])
    xmax = tf.math.reduce_min([bboxes1[..., 3], bboxes2[..., 3]])
    ### 2. Compute the following:
    # Area of the intersection
    intersection = tf.math.maximum(0, ymax - ymin) * tf.math.maximum(0, xmax - xmin)
    # Area of the first batch
    height = bboxes1[..., 2] - bboxes1[..., 0]
    width = bboxes1[..., 3] - bboxes1[..., 1]
    bboxes1_area = height * width
    # Area of the second batch
    height = bboxes2[..., 2] - bboxes2[..., 0]
    width = bboxes2[..., 3] - bboxes2[..., 1]
    bboxes2_area = height * width
    # Area of the union
    union = tf.cast(bboxes1_area + bboxes2_area - intersection, tf.dtypes.float32)
    ### 3. Return the IoU scores
    return tf.cast(intersection, tf.dtypes.float32) / (union + 1e-16)

To do so, you will need to:
* compare each bounding box with all the other bounding boxes in the set
* for each pair of bounding boxes, calculate the IoU and compare the scores
* if the IoU is above the threshold, keep the box with the highest score

#### NMS algorithm in Python

In [12]:
### From Udacity's `nms.py`

In [13]:
def nms(
        predictions: dict, 
        confidence_threshold: float=0.7, 
        iou_threshold: float=0.5
) -> List[Union[List[int], float]]:
    """Performs non-maximum suppression (Felzenszwalb et al., 2008).
    
    Credit: https://bit.ly/3xQeFom
    
    :param predictions: the dict instance containing the ground truth
        and predicted bounding box coordinates.
    :param confidence_threshold: float, the confidence threshold, any
        bounding box with score above this threshold is preserved.
    :param iou_threshold: float, intersection over union threshold, any
        bounding box with IoU score above this threshold is suppressed.
    :returns filtered: the nested list of thresholded bounding box coordinates
        and their corresponding confidence scores.
    """
    
    filtered = []
    # IMPLEMENT THIS FUNCTION
    
    ### 1. Discard all predictions below confidence threshold
    # Getting indices of bounding boxes w.r.t. confidence in descending order
    idxs = tf.argsort(predictions['scores'], direction='DESCENDING')
    # Sorting the bounding boxes and their scores
    scores = tf.gather(tf.cast(predictions['scores'], tf.dtypes.float32), idxs)
    bboxes = tf.gather(predictions['boxes'], idxs)
    # Keeping only the bounding boxes above the threshold
    bboxes = bboxes[scores > confidence_threshold]
    ### 2. For any remaining boxes
    while len(bboxes) >= 2:
        # Keep the bounding box with highest confidence score
        filtered.append([
            tf.squeeze(bboxes[0, :]).numpy().tolist(), tf.squeeze(scores[0]).numpy()])
        # Update the remaining bounding boxes and their scores
        other = bboxes[1:, :]
        scores = scores[1:]
        ### 3. Compute IoU scores between the bbox and all `m` other bboxes
        # `broadcast_to` repeats the bbox coordinates `m` times
        iou_scores = calculate_iou(
                        bboxes1=tf.broadcast_to(bboxes[0, :], other.shape),
                        bboxes2=other
        )
        # Keep bboxes whose IoU score is less than threshold
        idxs_keep = tf.where(iou_scores <= iou_threshold)
        bboxes = tf.gather(other, idxs_keep)
        scores = tf.gather(scores, idxs_keep)
    return filtered

You can run `python nms.py` to check your implementation.

#### Testing the NMS algorithm

In [14]:
### Fetching the test image filename
test_image = 'segment-1231623110026745648_480_000_500_000_with_camera_labels_38.png'

In [15]:
### From Udacity's `nms.py`

In [16]:
### Fetching the predicted data
with open('data/predictions_nms.json', 'r') as f:
    predictions = json.load(f)

In [17]:
predictions = tf.data.Dataset.from_tensors(predictions)

In [18]:
### Obtaining the 'picked' bounding boxes after performing NMS
output = nms(iter(predictions).get_next(), confidence_threshold=0.0, iou_threshold=0.5)

#### Evaluating our results

In [19]:
### From Udacity's `utils.py`

In [20]:
def check_results(output):
    truth = np.load(os.path.join(DIR_SRC, 'nms.npy'), allow_pickle=True)
    assert np.array_equal(truth, np.array(output, dtype="object")), 'The NMS implementation is wrong'
    print('The NMS implementation is correct!')

In [21]:
### Checking our results against Udacity's
with open(os.path.join(DIR_SRC, 'predictions_nms.json')) as f:
    preds_nms = json.load(f)
    preds_nms = tf.data.Dataset.from_tensors(preds_nms)
    check_results(nms(iter(preds_nms).get_next(), confidence_threshold=0.0, iou_threshold=0.5))

AssertionError: The NMS implementation is wrong

**UPDATE 2022-09-24 14:35 PT**: Looks like our updated `nms` function results aren't matching Udacity's. Let's investigate...

In [22]:
output_test = nms(iter(preds_nms).get_next(), confidence_threshold=0.0, iou_threshold=0.5)
output_test

[[[107, 107, 192, 148], 0.8147568], [[101, 90, 206, 141], 0.7962376]]

In [23]:
### Previewing the output results
np.array(output_test, dtype="object")

array([[list([107, 107, 192, 148]), 0.8147568],
       [list([101, 90, 206, 141]), 0.7962376]], dtype=object)

In [24]:
### Previewing the results from Udacity
np.load(os.path.join(DIR_SRC, 'nms.npy'), allow_pickle=True)

array([[list([107, 107, 192, 148]), 0.814756824762981],
       [list([101, 90, 206, 141]), 0.7962375654802895]], dtype=object)

So it seems as if this is a just floating-point precision error. Both the actual and expected results match in terms of bounding box coordinates and the confidence scores. However, the results from our `nms` function indicate that the confidence scores have been truncated (likely due to the `tf.dtypes.float32` format).

We can safely conclude that our `nms` method functions as intended.  

**UPDATE 2022-09-24 14:48 PT**: Something I noticed while refactoring `nms`... previously in the `calculate_iou` function I implemented in Numpy (see commit #[34a296d](https://github.com/jonathanloganmoran/ND0013-Self-Driving-Car-Engineer/blob/34a296d7ae16f1f2e97089137c000916e3482b06/1-Object-Detection-in-Urban-Environments/Exercises/1-5-1-Non-Maximum-Suppression/2022-09-21-Non-Maximum-Suppression.ipynb)), I was unintentionally calculating the area of intersection of _one_ and not _all_ boxes. In otherwords, I was only performing a `reduce_min` and `reduce_max` over the batch of bounding boxes (returning only one box's coordinates to compute the area of). 

I believe this is not the correct approach and that I should be instead computing the element-wise `min` and `max` for each coordinate. Therefore, the area of intersection between each bounding box can be calculated as intended. To fix this, I will need to change the `reduce_max`/`reduce_min` operations to their element-wise equivalents (`maximum`/`minimum`).

#### Visualising the results

In [20]:
### Fetching the ground-truth bounding boxes
with open(os.path.join(DIR_SRC, 'ground_truths.json'), 'r') as f:
    ground_truth = json.load(f)
ground_truth

[{'boxes': [[793, 1134, 1001, 1718],
   [737, 0, 898, 260],
   [763, 484, 878, 619],
   [734, 0, 1114, 277],
   [820, 1566, 974, 1914],
   [762, 951, 844, 1175],
   [748, 197, 803, 363]],
  'classes': [1, 1, 1, 1, 1, 1, 1],
  'filename': 'segment-1231623110026745648_480_000_500_000_with_camera_labels_38.png'}]

In [21]:
### From J. Moran's `2022-08-01-Data-Acquisition-Visualisation.ipynb`

In [22]:
def bbox_to_rectangle(bbox: List[int], **params) -> mpl.patches.Rectangle:
    """Renders bounding boxes as Matplotlib patches.
    
    Bounding box coordinate pairs form a single 
    orthogonal rectangle about the x-y plane.
    The upper-left coordinates are given by (y1, x1)
    and the lower-left coordinates by (y2, x2).
    
    Note that this assumes a bounding box coordinate ordering of:
        bbox := [y1, x1, y2, x2].
    
    :param bbox: list of bounding box coordinates
    :param **params: dict-like object of `matplotlib.patches.Rectangle`
        parameters, e.g., `linewidth`, `color`, and `linestyle`.
    :returns: matplotlib Rectangle patch instance
    """
    
    return patches.Rectangle(xy=(bbox[1], bbox[0]), 
                             width=(bbox[3]-bbox[1]), height=(bbox[2]-bbox[0]), 
                             **params)

In [23]:
# Bounding box colours
class_colourmap = {1:'cyan',        # Ground-truth 
                   2:'magenta',     # Predicted
                   3:'yellow'}      # NMS picked
default = 'red'

In [24]:
# Rectangle patch parameters
rect_params = {'fill': False,
               'edgecolor': default,
               'linewidth': 2,
               'linestyle': '--'
              }

In [ ]:
fig = plt.figure(figsize=(24,24))
image = Image.open(os.path.join(DIR_SRC, test_image))
ax = fig.add_subplot(111)
ax.imshow(image)

### Plot the ground-truth bounding boxes
for j, bbox in enumerate(ground_truth[0]['boxes']):
    # Get bounding box class
    cls_bbox = ground_truth[0]['classes'][j]
    # Update bbox color
    rect_params['edgecolor'] = class_colourmap.get(cls_bbox, 'red')
    ax.add_patch(bbox_to_rectangle(bbox, **rect_params))
### Plot the predicted bounding boxes
for j, bbox in enumerate(iter(predictions).get_next()['boxes'].numpy()):
    # Here we assume all bboxes are of same class;
    # instead use colour to indicate ground truth/predicted/picked
    cls_bbox = 2
    # Update bbox color
    rect_params['edgecolor'] = class_colourmap.get(cls_bbox, 'red')
    ax.add_patch(bbox_to_rectangle(bbox, **rect_params))
### Plot the picked bounding boxes preserved with NMS
for j, bbox in enumerate(output):
    # Here we assume all bboxes are of same class;
    # instead use colour to indicate ground truth/predicted/picked
    cls_bbox = 3
    # Update bbox color
    rect_params['edgecolor'] = class_colourmap.get(cls_bbox, 'red')
    ax.add_patch(bbox_to_rectangle(bbox[0], **rect_params))
ax.imshow(image)
plt.show()

Here we see that NMS didn't do such a good job of picking one bounding box per object ('picked' boxes shown in yellow). We'll have to explore this in more depth..

Wondering if it is possible that only the bounding boxes belonging to a single object should be passed into NMS to be thresholded...

### 2.2. Soft-NMS

In [28]:
def soft_nms(predictions: dict):
    """Soft-NMS algorithm as in Bodla et al., 2017.
    
    :param predictions: the dict instance containing the ground truth
        and predicted bounding box coordinates.
    :returns filtered: the list of thresholded bounding boxes and their
        computed Soft-NMS scores.
    """
    
    pass

## 3. Closing Remarks

##### Alternatives
* Use the [Soft-NMS](https://arxiv.org/abs/1704.04503) algorithm to handle occluded objects

##### Extensions of task
* Apply NMS to a object detection pipeline.

## 4. Future Work

- [ ] Compare NMS and Soft-NMS on images with occluded objects;
- [ ] Add NMS/Soft-NMS to an object detection pipeline.

## Credits

This assignment was prepared by Thomas Hossler et al., Winter 2021 (link [here](https://www.udacity.com/course/self-driving-car-engineer-nanodegree--nd0013)).


References

[1] Felzenszwalb, P. F., et al. Object Detection with Discriminatively Trained Part-Based Models. IEEE Transactions on Pattern Analysis and Machine Intelligence. 32(9):1627-1645. 2010. [doi:10.1109/TPAMI.2009.167](https://ieeexplore.ieee.org/document/5255236).

[2] Bodla, N. et al. Soft-NMS — Improving Object Detection With One Line of Code. arXiv. 2017. [doi:10.48550/ARXIV.1704.04503](https://arxiv.org/abs/1704.04503).



Further reading:
* Uijlings, J.R.R., et al. Selective Search for Object Recognition. International Journal of Computer Vision, 104:154–171. 2013. [doi:10.1007/s11263-013-0620-5](https://doi.org/10.1007/s11263-013-0620-5).

* Ren, S., et al., Faster R-CNN: Towards real-time object detection with region proposal networks. Advances in Neural Information Processing Systems, vol. 28. 2015. [doi:10.48550/ARXIV.1506.01497](https://arxiv.org/abs/1506.01497).



Helpful resources:
* [Selective Search for Object Recognition by S. Smith | CS231B at Stanford University](http://vision.stanford.edu/teaching/cs231b_spring1415/slides/ssearch_schuyler.pdf)